In [1]:
import numpy as np
import em_algorithm
from simulation_framework.simulate_competency import respondent_population
from simulation_framework.simulate_responses import response_simulation
from scipy.stats import multivariate_normal

C:\Users\Jesper\Documents\GitHub\Knowledge-Growth-Prediction\models


## Experiment 1: MIRT-2PL Parameter Recovery

In [2]:
from unittest import result


def mirt_param_recovery(sample_size) -> dict:
    latent_dimension = 2
    item_dimension = 4

    #Define Population
    cov = np.array([[1,0.2],
                    [0.2,1]])

    latent_distribution = multivariate_normal(mean=np.array([0,0]), cov=cov)
    population = respondent_population(latent_dimension=latent_dimension, latent_distribution=latent_distribution)

    #Define Test
    A = np.array([[0.5,0],
                  [2,0],
                  [0,0.4],
                  [0.7,0.6]])
    Q = np.array([[1,0],
                  [1,0],
                  [0,1],
                  [1,1]])
    delta = np.array([0, 0.5, 1, 0])
    early_item_parameters = {"A": A, "delta": delta, "q_matrix": Q, "item_dimension": item_dimension, "latent_dimension": latent_dimension}

    #Sample responses
    response_simulation_obj = response_simulation(population=population, early_item_params=early_item_parameters)
    sample = response_simulation_obj.sample(100)

    #Fit Parameters

    result_dict = {"sample": sample,
                   "early_item_parameters": early_item_parameters,
                   "latent_covariance": cov,
                   "estimated_parameters": None}
    return(result_dict)


In [9]:
import models
import em_algorithm
import pandas as pd

#sample data
exper_dict = mirt_param_recovery(30)
U = exper_dict["sample"]["early_responses"]
Q = exper_dict["early_item_parameters"]["q_matrix"]

#Initialize model
model = models.mirt_2pl(latent_dimension=2, item_dimension=4, A=Q, Q=Q, delta=np.ones(4), sigma=np.identity(2))

e_step = em_algorithm.e_step_ga_mml(incomplete_data=U, model=model)
m_step = em_algorithm.m_step_ga_mml(model)

em = em_algorithm.em_algo(e_step=e_step, m_step=m_step, model=model)
em.fit(U, max_iter=10)


EM Iteration 1
E-step
M-step
Maximize Q-0
Highest Current Fitness:
-272.52129424941415
Length of Population = 20
Highest Current Fitness:
-272.52129424941415
Length of Population = 30
Maximize the Q_i's
Highest Current Fitness:
-93.36098591254331
Length of Population = 20
Highest Current Fitness:
-93.36098591254331
Length of Population = 30
Highest Current Fitness:
-99.01250551107479
Length of Population = 20
Highest Current Fitness:
-95.21091866245443
Length of Population = 30
Highest Current Fitness:
-94.22755230412324
Length of Population = 40
Highest Current Fitness:
-94.22755230412324
Length of Population = 50
Highest Current Fitness:
-89.82273496577297
Length of Population = 20
Highest Current Fitness:
-100.51857032671234
Length of Population = 20
Step: 1: current parameter_diff: 3.1008918809435007, current data likelihood: 0.0
EM Iteration 2
E-step
M-step
Maximize Q-0
Highest Current Fitness:
-272.69487070453073
Length of Population = 20
Maximize the Q_i's
Highest Current Fitnes

### Estimeted Parameters

In [10]:
em.model.item_parameters

{'discrimination_matrix': array([[1.60041602, 0.        ],
        [3.88352924, 0.        ],
        [0.        , 1.04743765],
        [2.79398048, 0.60602585]]),
 'intercept_vector': array([3.51037676, 0.95974132, 1.26267253, 1.45433948]),
 'q_matrix': array([[1, 0],
        [1, 0],
        [0, 1],
        [1, 1]])}

In [11]:
em.model.person_parameters
#Die Kovarianz ist nicht symmetrisch

{'covariance': array([[1.        , 0.41384366],
        [0.41384366, 1.        ]])}